# TODO: 

- [ ] Need to document notebook

# Global Fishing Watch

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
import glob
import dask.dataframe as dd
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import numpy as np

import warnings
warnings.filterwarnings('ignore')

from ipywidgets import widgets
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


import timeit
start_time = timeit.default_timer()

from sklearn.datasets import fetch_species_distributions
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity
from sklearn.datasets.base import Bunch


from shapely.geometry import Point
from shapely.geometry import MultiLineString,mapping
from shapely.ops import polygonize
import geopandas as gpd

from h3 import h3

In [ ]:
def gfwDataNew():
    csvs = 'Global_Fishing_Watch/*/*.csv'
    All_GFW_Grid = dd.read_csv(csvs)
    
    All_GFW_Grid.lat_bin = All_GFW_Grid.lat_bin / 10
    All_GFW_Grid.lon_bin = All_GFW_Grid.lon_bin / 10
    
    return All_GFW_Grid 

In [ ]:
All_GFW_Grid = gfwDataNew()
mask = ((All_GFW_Grid['date'] >= '2018-08-02') & 
        (All_GFW_Grid['date'] <= '2018-12-28' ))
All_GFW_Grid = All_GFW_Grid.loc[mask]

In [ ]:
len(All_GFW_Grid.index)

In [ ]:
All_GFW_Grid.compute().head()

# Beneath the Waves

In [ ]:
Beneath_the_Waves = dd.read_csv('Beneath_The_Waves/Gallagher_BTW_Tracks.csv')
Beneath_the_Waves['dt'] = dd.to_datetime(Beneath_the_Waves['dt'])
Beneath_the_Waves = Beneath_the_Waves.set_index('dt')

In [ ]:
Beneath_the_Waves.head(2)

In [ ]:
len(Beneath_the_Waves.index)

In [ ]:
start_btw = Beneath_the_Waves.index.min().compute()
end_btw = Beneath_the_Waves.index.max().compute()

In [ ]:
print(start_btw, '-', end_btw)

In [ ]:
Beneath_the_Waves['class'].compute().unique()

In [ ]:
Beneath_the_Waves = Beneath_the_Waves[Beneath_the_Waves['class'].isin(['0','1','2', '3'])]

In [ ]:
len(Beneath_the_Waves.index)

In [ ]:
All_GFW_Grid['lat_centroid'] = All_GFW_Grid.loc[:]['lat_bin'] + 0.05
All_GFW_Grid['lon_centroid'] = All_GFW_Grid.loc[:]['lon_bin'] + 0.05

In [ ]:
first_part = All_GFW_Grid.compute().head()

In [ ]:
def convert_to_hexagon(df, lat_label, lon_label, resolution=7):
    x = df[[lat_label, lon_label]].values.tolist()
    l = []
    for i in x:
        l.append(h3.geo_to_h3(i[0], i[1], resolution))
    return l

In [ ]:
All_GFW_Grid['h3'] = All_GFW_Grid.map_partitions(convert_to_hexagon, 
                                                 lat_label='lat_centroid', 
                                                 lon_label='lon_centroid', 
                                                 resolution=7)

In [ ]:
Beneath_the_Waves['h3'] = Beneath_the_Waves.map_partitions(convert_to_hexagon,
                                                           lat_label='lat', 
                                                           lon_label='lon',
                                                           resolution=7)

In [ ]:
intersections = dd.merge(All_GFW_Grid, Beneath_the_Waves, on='h3').compute()

In [ ]:
pd.unique(Beneath_the_Waves['ptt'])

In [ ]:
intersections

In [ ]:
len(intersections.index)

In [ ]:
from keplergl import KeplerGl

map_2 = KeplerGl(height=800, data={"intersections": intersections, 
                                  "beneath_the_waves": Beneath_the_Waves.compute()})
map_2